In [4]:
import pandas as pd

df = pd.read_csv('data_bert1.csv')

df.head()

,category,text,clean_text,words_removed
0,arts,you are using an older browser version. please...,you are an older browser version . please use ...,50
1,arts,on 31 march two of classical music s most acco...,on 31 march two of classical music s most acco...,156
2,arts,bpt after a year of being locked away at home ...,after a year of being locked away at home the ...,98
3,arts,pilot uninjured plane hit sandbar while landin...,pilot uninjured plane hit while landing . a fl...,22
4,arts,colleen distin photo by facebook toronto sun ....,colleen photo by sun . a lost wallet been retu...,47


In [5]:
df["category"].unique()

array(['arts', 'crime', 'disaster', 'economy', 'education',
       'environmental', 'health', 'humaninterest', 'labour', 'lifestyle',
       'other', 'politics', 'religion', 'science', 'social', 'sport',
       'unrest', 'weather'], dtype=object)

In [22]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from nltk import sent_tokenize
import nltk
from collections import Counter
import nltk
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier


weighted_dict = {
    "arts": {
        "artist": 1.0,
        "exhibition": 0.9,
        "gallery": 0.9,
        "sculpture": 0.8,
        "performance": 0.8,
        "museum": 0.7,
        "creative": 0.6,
        "painting": 0.7,
    },
    "crime": {
        "crime": 1.0,
        "theft": 0.9,
        "murder": 0.95,
        "police": 0.8,
        "arrest": 0.85,
        "fraud": 0.75,
        "investigation": 0.7,
        "law": 0.6,
    },
    "disaster": {
        "earthquake": 0.95,
        "flood": 0.9,
        "hurricane": 0.9,
        "disaster": 1.0,
        "emergency": 0.8,
        "rescue": 0.75,
        "casualty": 0.7,
        "evacuation": 0.7,
    },
    "economy": {
        "gdp": 0.9,
        "inflation": 0.95,
        "market": 0.85,
        "trade": 0.8,
        "stock": 0.75,
        "recession": 0.9,
        "currency": 0.7,
        "deficit": 0.7,
    },
    "education": {
        "school": 0.9,
        "student": 0.95,
        "teacher": 0.85,
        "university": 0.8,
        "curriculum": 0.7,
        "tuition": 0.7,
        "degree": 0.6,
        "exam": 0.6,
    },
    "environmental": {
        "pollution": 0.95,
        "climate": 0.9,
        "recycling": 0.8,
        "sustainability": 0.85,
        "carbon": 0.9,
        "emissions": 0.85,
        "wildlife": 0.7,
        "conservation": 0.75,
    },
    "health": {
        "hospital": 0.9,
        "disease": 0.95,
        "vaccine": 0.85,
        "patient": 0.8,
        "medical": 0.85,
        "pandemic": 1.0,
        "treatment": 0.7,
        "mental": 0.6,
    },
    "humaninterest": {
        "community": 0.8,
        "story": 0.9,
        "charity": 0.75,
        "volunteer": 0.7,
        "personal": 0.85,
        "inspiration": 0.7,
        "family": 0.6,
        "achievement": 0.65,
    },
    "labour": {
        "union": 0.9,
        "employment": 0.8,
        "strike": 0.85,
        "wage": 0.9,
        "contract": 0.7,
        "workers": 0.95,
        "benefits": 0.6,
        "overtime": 0.6,
    },
    "lifestyle": {
        "fashion": 0.9,
        "travel": 0.85,
        "diet": 0.8,
        "hobby": 0.7,
        "wellness": 0.75,
        "luxury": 0.6,
        "trend": 0.7,
        "fitness": 0.6,
    },
    "other": {
        "miscellaneous": 0.5,
        "unknown": 0.3,
        "general": 0.4,
        "unclassified": 0.3,
        "notice": 0.2,
    },
    "politics": {
        "government": 0.95,
        "election": 0.9,
        "policy": 0.85,
        "diplomacy": 0.8,
        "senate": 0.75,
        "vote": 0.9,
        "democracy": 0.7,
        "law": 0.6,
    },
    "religion": {
        "faith": 0.9,
        "temple": 0.8,
        "prayer": 0.85,
        "ritual": 0.75,
        "church": 0.8,
        "islam": 0.7,
        "christianity": 0.7,
        "worship": 0.7,
    },
    "science": {
        "research": 0.95,
        "experiment": 0.9,
        "technology": 0.85,
        "data": 0.8,
        "discovery": 0.85,
        "physics": 0.75,
        "biology": 0.7,
        "innovation": 0.7,
    },
    "social": {
        "equality": 0.9,
        "welfare": 0.85,
        "activism": 0.8,
        "diversity": 0.75,
        "rights": 0.9,
        "justice": 0.7,
        "community": 0.6,
        "protest": 0.65,
    },
    "sport": {
        "athlete": 0.95,
        "match": 0.9,
        "tournament": 0.85,
        "score": 0.8,
        "league": 0.75,
        "olympics": 0.7,
        "training": 0.6,
        "championship": 0.8,
    },
    "unrest": {
        "protest": 0.9,
        "riot": 0.95,
        "clash": 0.85,
        "violence": 0.9,
        "tension": 0.8,
        "conflict": 0.75,
        "demonstration": 0.7,
        "angry": 0.6,
    },
    "weather": {
        "forecast": 0.9,
        "storm": 0.95,
        "temperature": 0.8,
        "rain": 0.75,
        "drought": 0.7,
        "sunny": 0.6,
        "climate": 0.6,
        "hurricane": 0.7,
    },
}

x = df["clean_text"]
y = df["category"]
# Train the Random Forest classifier
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_vectorized, y_train)

y_pred = rf.predict(X_test_vectorized)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

def dictionary_score(sentence, weighted_dict):
    """
    Computes a score for each class based on the presence of weighted keywords.
    """
    words = sentence.lower().split()
    scores = {cls: 0 for cls in weighted_dict}
    for cls in weighted_dict:
        for word in words:
            scores[cls] += weighted_dict[cls].get(word, 0)
    return scores

def classifier_predict(sentence):
    X_sentence = vectorizer.transform([sentence])
    probas = rf.predict_proba(X_sentence)[0]
    return dict(zip(rf.classes_, probas))

def combine_predictions(clf_pred, dict_scores, alpha=0.5):
    """
    Combine classifier probabilities and dictionary scores.
    The parameter alpha weights the classifier output.
    """
    combined = {}
    for cls in clf_pred:
        combined[cls] = alpha * clf_pred[cls] + (1 - alpha) * dict_scores.get(cls, 0)
    return combined

def classify_text(text, alpha=0.5):
    """
    Tokenize the text into sentences, combine classifier and dictionary scores,
    and return the final predicted class based on majority vote.
    """
    sentences = sent_tokenize(text)
    sentence_labels = []
    
    for sentence in sentences:
        clf_pred = classifier_predict(sentence)
        dict_scores = dictionary_score(sentence, weighted_dict)
        combined_scores = combine_predictions(clf_pred, dict_scores, alpha)
        # Select the class with the highest combined score for the sentence
        sentence_label = max(combined_scores, key=combined_scores.get)
        sentence_labels.append(sentence_label)
    
    # Final classification by taking the most common class (majority vote)
    final_label = Counter(sentence_labels).most_common(1)[0][0]
    return final_label

# Example usage:
text_row = ("The gallery showcased a variety of paintings and sculptures. "
            "A recent experiment in the laboratory produced groundbreaking results.")
final_prediction = classify_text(text_row, alpha=0.6)
print("Final predicted class:", final_prediction)

Accuracy: 0.46458923512747874
Final predicted class: arts


In [24]:
def dictionary_score(sentence, weighted_dict):
    """
    Computes a score for each class based on the presence of weighted keywords.
    """
    words = sentence.lower().split()
    scores = {cls: 0 for cls in weighted_dict}
    for cls in weighted_dict:
        for word in words:
            scores[cls] += weighted_dict[cls].get(word, 0)
    return scores

def classifier_predict(sentence):
    """
    Uses the trained Random Forest and TF-IDF vectorizer to get class probabilities.
    """
    X_sentence = vectorizer.transform([sentence])
    probas = rf.predict_proba(X_sentence)[0]
    return dict(zip(rf.classes_, probas))

def combine_predictions(clf_pred, dict_scores, alpha=0.5):
    """
    Combine classifier probabilities and dictionary scores.
    The parameter alpha weights the classifier output.
    """
    combined = {}
    for cls in clf_pred:
        combined[cls] = alpha * clf_pred[cls] + (1 - alpha) * dict_scores.get(cls, 0)
    return combined

def classify_text(text, alpha=0.5):
    """
    Tokenize the text into sentences, combine classifier and dictionary scores,
    and return the final predicted class based on majority vote.
    """
    sentences = sent_tokenize(text)
    sentence_labels = []
    
    for sentence in sentences:
        clf_pred = classifier_predict(sentence)
        dict_scores = dictionary_score(sentence, weighted_dict)
        combined_scores = combine_predictions(clf_pred, dict_scores, alpha)
        
        # Select the class with the highest combined score for the sentence
        sentence_label = max(combined_scores, key=combined_scores.get)
        sentence_labels.append(sentence_label)
    
    # Final classification by taking the most common class (majority vote)
    final_label = Counter(sentence_labels).most_common(1)[0][0]
    return final_label


df["predicted_category"] = df["clean_text"].apply(lambda x: classify_text(x, alpha=0.5))


In [25]:
accuracy = accuracy_score(df["category"], df["predicted_category"])
print("Overall Accuracy:", accuracy)

Overall Accuracy: 0.24603174603174602


In [26]:
x = df["clean_text"]
y = df["predicted_category"]
# Train the Random Forest classifier
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

rf = RandomForestClassifier(n_estimators=300, random_state=42)
rf.fit(X_train_vectorized, y_train)

y_pred = rf.predict(X_test_vectorized)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.6515580736543909
